In [3]:
from ibl_pipeline.analyses import behavior as behavior_analyses
from ibl_pipeline import behavior as behavior
from ibl_pipeline import subject, acquisition
from ibl_pipeline.utils import psychofit as psy
import numpy as np
import datetime
import datajoint as dj
import json
import seaborn as sns
import pandas as pd
import plotly
import plotly.graph_objs as go
import statsmodels.stats.proportion as smp
plotly.offline.init_notebook_mode(connected=True)

In [6]:
subj_uuid = (subject.Subject & 'subject_nickname="IBL-T3"').fetch1('subject_uuid')
key = {'subject_uuid': subj_uuid,
   'session_date': datetime.date(2019, 4, 18)}
sessions = behavior_analyses.BehavioralSummaryByDate.PsychResults & key

data = []

for session in sessions.fetch('KEY'):
    contrasts, prob_right, prob_left, threshold, bias, lapse_low, lapse_high, n_trials, n_trials_right = \
        (sessions & session).fetch1(
            'signed_contrasts', 'prob_choose_right', 'prob_left', 'threshold', 'bias', 
            'lapse_low', 'lapse_high', 'n_trials_stim', 'n_trials_stim_right')
    pars = [bias, threshold, lapse_low, lapse_high]
    contrasts = contrasts * 100
    contrasts_fit = np.arange(-100, 100)
    prob_right_fit = psy.erf_psycho_2gammas(pars, contrasts_fit)
    ci = smp.proportion_confint(
    n_trials_right, n_trials, alpha=0.032, method='normal') - prob_right
    
    if prob_left == 0.2:
        curve_color = 'orange'
    elif prob_left == 0.5:
        curve_color = 'black'
    elif prob_left == 0.8:
        curve_color = 'cornflowerblue'
    else:
        continue

    behavior_data = go.Scatter(
        x = contrasts.tolist(),
        y = prob_right.tolist(),
        error_y = dict(
                    type='data',
                    array=ci[0].tolist(),
                    arrayminus=np.negative(ci[1]).tolist(),
                    visible=True
            ),
        marker = dict(
                size = 6,
                color = curve_color),
        mode = 'markers',
        name = f'p_left = {prob_left}, data'
    )

    behavior_fit = go.Scatter(
        x = contrasts_fit.tolist(),
        y = prob_right_fit.tolist(),
        name = f'p_left = {prob_left} model fits',
        marker = dict(color=curve_color)
    )

    data.append(behavior_data)
    data.append(behavior_fit)

layout = go.Layout(
    width=630,
    height=400,
    title='Psychometric Curve', 
    xaxis={'title':'Contrast(%)'}, 
    yaxis={'title':'Probability choosing right',
           'range': [-0.05, 1.05]},
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [ ]:
f = open("psych_results_date.json","w")
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()